In [1]:
import pandas as pd

spillover_df = pd.read_csv("Spillover_simulator%3A_port-level_impact.csv")
spillover_df.head()

,from_portid,from_portname,from_country,from_iso3,from_lat,from_lon,to_portid,to_portname,to_country,to_iso3,to_lat,to_lon,average_transit_days,daily_capacity_at_risk,relative_capacity_at_risk,ObjectId
0,port0,Abbot Point,Australia,AUS,-19.893755,148.07762,port1010,Porto Pecem,Brazil,BRA,-3.538117,-38.804692,67.1,19.462076,0.000103,1
1,port0,Abbot Point,Australia,AUS,-19.893755,148.07762,port1048,Bahia Blanca,Argentina,ARG,-38.782142,-62.283465,61.4,31.339187,0.000208,2
2,port0,Abbot Point,Australia,AUS,-19.893755,148.07762,port1065,Busan,Korea,KOR,35.089551,128.938595,1.9,74.600421,0.000035,3
3,port0,Abbot Point,Australia,AUS,-19.893755,148.07762,port1067,Pyeongtaek,Korea,KOR,36.975128,126.784628,27.5,655.013560,0.001047,4
4,port0,Abbot Point,Australia,AUS,-19.893755,148.07762,port1069,Qingdao,China,CHN,36.026779,120.222618,33.0,592.748760,0.000230,5


In [2]:
Aracaju = spillover_df[
    (spillover_df["from_portname"] == "Port De Aracaju")]

Aracaju.head()

,from_portid,from_portname,from_country,from_iso3,from_lat,from_lon,to_portid,to_portname,to_country,to_iso3,to_lat,to_lon,average_transit_days,daily_capacity_at_risk,relative_capacity_at_risk,ObjectId
219235,port941,Port De Aracaju,Brazil,BRA,-10.825369,-36.940532,port487,IJmuiden,The Netherlands,NLD,52.476716,4.617278,47.0,16.408792,0.000064,219236
219236,port941,Port De Aracaju,Brazil,BRA,-10.825369,-36.940532,port1000,Porto De Maceio,Brazil,BRA,-9.679541,-35.723009,3.1,73.276667,0.002724,219237
219237,port941,Port De Aracaju,Brazil,BRA,-10.825369,-36.940532,port1004,Ravenna,Italy,ITA,44.459445,12.252757,23.0,14.148108,0.000076,219238
219238,port941,Port De Aracaju,Brazil,BRA,-10.825369,-36.940532,port1009,Porto Grande,Cabo Verde,CPV,16.894221,-24.995660,37.8,14.178849,0.000260,219239
219239,port941,Port De Aracaju,Brazil,BRA,-10.825369,-36.940532,port1010,Porto Pecem,Brazil,BRA,-3.538117,-38.804692,2.3,14.252713,0.000076,219240


In [3]:
# Filters for exports from Aratu
aratu_exports = spillover_df[
    spillover_df["from_portname"] == "Aratu"
]

# Exclude any self-routes (if they exist)
aratu_exports = aratu_exports[
    aratu_exports["to_portname"] != "Aratu"
]

print("Aratu-Aratu routes (should be empty):", len(aratu_exports[aratu_exports["to_portname"] == "Aratu"]))

# Total downstream capacity at risk
downstream_daily_loss = aratu_exports["daily_capacity_at_risk"].sum()

print("Total downstream capacity at risk (1-day shutdown):",
      downstream_daily_loss, "metric tons")

print("Total downstream capacity at risk (1-week shutdown):",
      downstream_daily_loss * 7, "metric tons")

Aratu-Aratu routes (should be empty): 0
Total downstream capacity at risk (1-day shutdown): 32482.868387877075 metric tons
Total downstream capacity at risk (1-week shutdown): 227380.07871513953 metric tons


In [4]:
aratu_exports = spillover_df[
    (spillover_df["from_portname"] == "Aratu") &
    (spillover_df["to_portname"] != "Aratu")
]

# 2) Aggregate daily capacity at risk by destination
downstream_risk = (
    aratu_exports
    .groupby(["to_portname", "to_country"])["daily_capacity_at_risk"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# 3) Add one-week spillover column
downstream_risk["weekly_capacity_at_risk"] = (
    downstream_risk["daily_capacity_at_risk"] * 7
)
total_daily = downstream_risk["daily_capacity_at_risk"].sum()

downstream_risk["percent_of_total"] = (
    downstream_risk["daily_capacity_at_risk"] / total_daily * 100
)


import pandas as pd

pd.set_option("display.expand_frame_repr", False)
print(downstream_risk.head(20))



                     to_portname       to_country  daily_capacity_at_risk  weekly_capacity_at_risk  percent_of_total
0                         Santos           Brazil             5688.183746             39817.286224         17.511335
1               Port De Salvador           Brazil             3125.237563             21876.662940          9.621187
2                      Paranagua           Brazil             1629.326037             11405.282256          5.015955
3                 Porto De Suape           Brazil             1489.363636             10425.545450          4.585074
4   Las Palmas (de Gran Canaria)            Spain             1294.726849              9063.087944          3.985876
5                        Vitoria           Brazil             1005.516835              7038.617844          3.095530
6                       Zhoushan            China              681.657320              4771.601243          2.098513
7                     Rio Grande           Brazil              6

In [ ]:
# 1) Filter import routes into Aratu
aratu_imports = spillover_df[
    (spillover_df["to_portname"] == "Aratu") &
    (spillover_df["from_portname"] != "Aratu")
]

# 2) Aggregate daily capacity at risk by origin port
upstream_risk = (
    aratu_imports
    .groupby(["from_portname", "from_country"])["daily_capacity_at_risk"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# 3) Add one-week spillover column
upstream_risk["weekly_capacity_at_risk"] = (
    upstream_risk["daily_capacity_at_risk"] * 7
)

# 4) Add percent of total upstream spillover
total_upstream_daily = upstream_risk["daily_capacity_at_risk"].sum()

upstream_risk["percent_of_total"] = (
    upstream_risk["daily_capacity_at_risk"] / total_upstream_daily * 100
)

# 5) Clean display
pd.set_option("display.expand_frame_repr", False)
print(upstream_risk.head(20))

                   from_portname        from_country  daily_capacity_at_risk  weekly_capacity_at_risk  percent_of_total
0               Port De Salvador              Brazil             9832.413595             68826.895164         32.890382
1                        Houston       United States             1691.832268             11842.825873          5.659344
2                         Itaqui              Brazil              929.099263              6503.694843          3.107928
3                         Santos              Brazil              772.689913              5408.829392          2.584723
4   Las Palmas (de Gran Canaria)               Spain              722.332138              5056.324969          2.416271
5                 Corpus Christi       United States              689.636212              4827.453485          2.306900
6                      Cartagena               Spain              689.516935              4826.618545          2.306501
7                      Paranagua        

In [ ]:
shutdown_ports = ["Aratu", "Port De Salvador"]

# 1) Filter export routes from either shutdown port
combined_exports = spillover_df[
    (spillover_df["from_portname"].isin(shutdown_ports)) &
    (~spillover_df["to_portname"].isin(shutdown_ports))   # exclude destinations to either port
]

# 2) Aggregate daily capacity at risk by destination
downstream_risk = (
    combined_exports
    .groupby(["to_portname", "to_country"])["daily_capacity_at_risk"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# 3) Add one-week spillover column
downstream_risk["weekly_capacity_at_risk"] = (
    downstream_risk["daily_capacity_at_risk"] * 7
)

# 4) Add percent of total spillover
total_daily = downstream_risk["daily_capacity_at_risk"].sum()

downstream_risk["percent_of_total"] = (
    downstream_risk["daily_capacity_at_risk"] / total_daily * 100
)

# 5) Clean display
import pandas as pd
pd.set_option("display.expand_frame_repr", False)

print(downstream_risk.head(20))

# Export full upstream risk table to CSV
out_path = "downstream_risk_aratu_salvador.csv"
downstream_risk.to_csv(out_path, index=False, encoding="utf-8")

print(f"Saved: {out_path} (rows={len(upstream_risk)})")

print("Total downstream daily capacity at risk:", downstream_risk["daily_capacity_at_risk"].sum())


                     to_portname       to_country  daily_capacity_at_risk  weekly_capacity_at_risk  percent_of_total
0                         Santos           Brazil            14598.838385            102191.868692         14.066552
1                  Madre De Deus           Brazil            13985.916217             97901.413521         13.475978
2                 Porto De Suape           Brazil            10278.937971             71952.565796          9.904159
3                    Porto Pecem           Brazil             8922.998073             62460.986512          8.597658
4                 Rio De Janeiro           Brazil             5453.451120             38174.157840          5.254614
5                      Cartagena         Colombia             4726.202078             33083.414546          4.553881
6          Tangier-Mediterranean          Morocco             3604.580581             25232.064069          3.473154
7   Las Palmas (de Gran Canaria)            Spain             21

### UPSTREAM RISK FOR BOTH PORTS

In [ ]:



shutdown_ports = ["Aratu", "Port De Salvador"]

# 1) Filter import routes into either shutdown port
combined_imports = spillover_df[
    (spillover_df["to_portname"].isin(shutdown_ports)) &
    (~spillover_df["from_portname"].isin(shutdown_ports))  # exclude self/within-shutdown routes
]

# 2) Aggregate daily capacity at risk by origin
upstream_risk = (
    combined_imports
    .groupby(["from_portname", "from_country"])["daily_capacity_at_risk"]
    .sum()
    .sort_values(ascending=False)
    .reset_index()
)

# 3) Add one-week spillover column
upstream_risk["weekly_capacity_at_risk"] = (
    upstream_risk["daily_capacity_at_risk"] * 7
)

# 4) Add percent of total upstream spillover
total_upstream_daily = upstream_risk["daily_capacity_at_risk"].sum()

upstream_risk["percent_of_total"] = (
    upstream_risk["daily_capacity_at_risk"] / total_upstream_daily * 100
)

# 5) Clean display
import pandas as pd
pd.set_option("display.expand_frame_repr", False)

print(upstream_risk.head(20))

# Export full upstream risk table to CSV
out_path = "upstream_risk_aratu_salvador.csv"
upstream_risk.to_csv(out_path, index=False, encoding="utf-8")

print(f"Saved: {out_path} (rows={len(upstream_risk)})")

print("Total upstream daily capacity at risk:", upstream_risk["daily_capacity_at_risk"].sum())


                   from_portname   from_country  daily_capacity_at_risk  weekly_capacity_at_risk  percent_of_total
0                 Rio De Janeiro         Brazil            20119.360956            140835.526689         20.184143
1                 Porto De Suape         Brazil             9097.300257             63681.101799          9.126592
2                         Santos         Brazil             6532.907679             45730.353756          6.553943
3          Tangier-Mediterranean        Morocco             5859.574313             41017.020189          5.878441
4                  Madre De Deus         Brazil             4978.270666             34847.894662          4.994300
5               Porto de Itaguai         Brazil             3119.220591             21834.544137          3.129264
6                  Sao Sebastiao         Brazil             3082.823011             21579.761079          3.092749
7                        Houston  United States             3068.201711         